<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# loading data from a csv fle
url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/R_pop_stat.csv'
data = pd.read_csv(url)

# preprocessing loaded
data = data.dropna(subset=['AGE'])
data = data[(data['AGE'] >= 20) & (data['AGE'] <= 50)]

data_dummy = pd.get_dummies(data, columns=['SEX', 'SCHL', 'WORK'], drop_first=True)

In [0]:
# [1]

mean_income = data['INCOME'].mean()
std_income = data['INCOME'].std()

ul = mean_income + 3 * std_income
ll = mean_income - 3 * std_income

outlier = data[(data['INCOME'] > ul) | (data['INCOME'] < ll)]
range = outlier['INCOME'].max() - outlier['INCOME'].min()

print(f'Answer [1] : Range = {range:.0f}')

data = data[(data['INCOME'] <= ul) & (data['INCOME'] >= ll)]
data_dummy = data_dummy[(data_dummy['INCOME'] <= ul) & (data_dummy['INCOME'] >= ll)]

Answer [1] : Range = 3325000


In [0]:
# [2]

import math

data2 = data[data['INCOME'] >= data['INCOME'].quantile(q=0.75)]
corr = data2['INCOME'].corr(data2['AGE'], method='spearman')

print(f'Answer [2] : Correlation = {math.floor(corr * 1000) / 1000}')

Answer [2] : Correlation = 0.115


In [0]:
# [3]

from scipy.stats import chi2_contingency
import numpy as np

data3 = data
data3['High'] = data3['INCOME'].apply(lambda x: 1 if x >= 25000 else 0)
temp = data3.groupby(['High', 'WORK']).count()
temp.reset_index(level=['High', 'WORK'], inplace=True)
temp = temp[['High', 'WORK', 'AGE']]

cont = pd.DataFrame({'Low': list(temp['AGE'][:4]),
                     'High': list(temp['AGE'][4:])},
                     index=['공무원', '기업', '비영리단체', '자영업'])

stat, p_value, dof, expected = chi2_contingency(cont)

print(f'Answer [3] : P-value = {p_value:.5f}')

Answer [3] : P-value = 0.00044


In [0]:
# [4]

import math
from sklearn.linear_model import LinearRegression

data4 = data_dummy[data_dummy['XGRP'] <= 7]

cols = list(data4.columns)
cols.remove('INCOME')
cols.remove('XGRP')

train_x = data4[cols]
train_y = data4['INCOME'].apply(lambda x: math.log10(x))

lr = LinearRegression()
lr.fit(train_x, train_y)

test = pd.DataFrame({'AGE': 38, 'SEX_여': 0, 'SCHL_박사': 0, 'SCHL_석사': 0,
                     'SCHL_전문학사': 0, 'SCHL_중졸이하': 0, 'SCHL_학사': 1,
                     'WORK_기업': 0, 'WORK_비영리단체': 1, 'WORK_자영업': 0},
                     index={0})

pred = lr.predict(test)

print(f'Answer [4] : Prediction = {round(10 ** (pred[0]), -2):.0f}')

Answer [4] : Prediction = 47400


In [0]:
# [5]

from sklearn.metrics import mean_squared_error

data5 = data_dummy[data_dummy['XGRP'] > 7]

test_x = data5[cols]
test_y = data5['INCOME'].apply(lambda x: math.log10(x))

preds = lr.predict(test_x)
mse = mean_squared_error(test_y, preds)
rmse = np.sqrt(mse)

print(f'Answer [5] : RMSE = {rmse:.2f}')

Answer [5] : RMSE = 0.29
